# 줄리아를 생각하다(Think julia)

## Chapter 16. 구조체와 함수

사용자 정의 객체를 인수로 받아 반환값으로 내놓는 함수 작성

함수형 프로그래밍(functional programming)과 계획된 개발(designed development)의 두 가지 새로운 개발 계획

### 16.1 시간

**예제1)** 어떤 시점의 시각을 기록하는 구조체 MyTime 작성

In [1]:
"""
Represents the time of day.

fields: hour, minute, second
"""

struct MyTime
    hour
    minute
    second
end

In [18]:
time = MyTime(11, 59, 5)

MyTime(11, 59, 5)

In [28]:
function printtime(time)
    h = time.hour
    m = time.minute
    s = time.second
    
    println("$h:$m:$s")
end

printtime(time)

11:59:5


In [21]:
println(time.hour,":",time.minute,":",time.second)

11:59:5


In [9]:
using Printf

In [24]:
@printf "%02d : %02d : %02d" time.hour time.minute time.second

11 : 59 : 05

### 16.2 순수 함수

* 순수 함수(pure function) : 값만 반환하는 함수

* 변경자(modifier) : 함수의 인수로 받는 객체를 수정하는 함수

* 프로토타이핑(prototype and patch) : 단순한 버전 -> 테스트해보면서 에러를 수정하는 방식

In [25]:
function addtime(t1, t2)
    MyTime(t1.hour + t2.hour, t1.minute + t2.minute, t1.second + t2.second)
end

addtime (generic function with 1 method)

In [29]:
start = MyTime(9, 45, 0);
duration = MyTime(1, 35, 0);
done = addtime(start, duration);
printtime(done)

10:80:0


In [32]:
# 수정된 버전의 addtime

function addtime(t1, t2)
    second = t1.second + t2.second
    minute = t1.minute + t2.minute
    hour= t1.hour + t2.hour
    
    if second >= 60
        second -= 60
        minute += 1
    end
    
    if minute >= 60
        minute -= 60
        hour+= 1
    end
    MyTime(hour, minute, second)
end

addtime (generic function with 1 method)

In [31]:
done = addtime(start, duration);
printtime(done)

11:20:0


### 16.3 변경자

In [33]:
function increment!(time, seconds)
    time.second += seconds
    if time.second >= 60 
        time.second -= 60
        time.minute += 1
    end
    if time.minute >= 60
        time.minute -= 60
        time.hour += 1
    end
end

increment! (generic function with 1 method)

### 16.4 시제품 만들기 vs 계획하기

문제를 일반화하여 **계획된 개발(designed development)** 을 수행

* 문제에 대한 수준 높은 통찰을 바탕으로 프로그래밍을 더 쉽게 만드는 방법

* 시간을 초 단위 정수로 바꾸는 함수 작성

In [42]:
function timetoint(time)
    minutes = time.hour * 60 + time.minute
    seconds = minutes *60 + time.second
end

timetoint (generic function with 1 method)

* 초 단위 정수를 다시 시간으로 변경하는 함수 작성

In [43]:
function inttotime(seconds)
    (minutes, second) = (seconds ÷ 60 , seconds %60)
    hour, minute = (minutes ÷ 60 , minutes %60)
    MyTime(hour, minute, second)
end

inttotime (generic function with 1 method)

* 초 단위 정수로 변환하여 두 시간을 합한 후 다시 시간으로 변경하는 함수를 리턴하는 addtime 함수 작성

In [44]:
function addtime(t1, t2)
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
end

addtime (generic function with 1 method)

### 16.5 디버깅

* **불변식(invariant)** : 요구 사항을 나타내는 조건식

    - 예) MyTime 객체는 minute와 second가 0 이상 60 미만이고, hour가 0보다 크거나 같아야 하며, second는 소수점 아래 허용 필요 등

**예시1)** MyTime 객체를 받아서 불변식 위반 여부를 체크하는 함수 isvalidtime 작성

In [45]:
function isvalidtime(time)
    if time.hour < 0 || time.minute < 0 || time.second < 0
        return false
    end
    if time.minute >= 60 || time.second >= 60
        return false
    end
    true
end

isvalidtime (generic function with 1 method)

* 각 함수의 앞쪽에 인수가 정상적인지 체크하는 코드 삽입

In [49]:
function addtime(t1, t2)
    if !isvalidtime(t1) || !isvalidtime(t2)
        error("invalid MyTime object in add_time")
    end
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
end

addtime (generic function with 1 method)

* 다른 방법으로는 @assert 매크로를 사용할 수 있음

* **@assert** 매크로는 조건식이 false인 경우에 예외를 출력함

    - @assert 매크로를 사용하면 오류 체크를 위한 코드와 일반 코드를 쉽게 구분할 수 있음

In [52]:
function addtime2(t1, t2)
    @assert(isvalidtime(t1) && isvalidtime(t2), "invalid MyTime object in add_time")
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
end

addtime2 (generic function with 1 method)

In [50]:
addtime(start, duration)

MyTime(11, 20, 0)

In [53]:
addtime2(start, duration)

MyTime(11, 20, 0)

In [54]:
addtime(MyTime(9, 45, 0), MyTime(1, 80, 0))

LoadError: invalid MyTime object in add_time

In [55]:
addtime2(MyTime(9, 45, 0), MyTime(1, 80, 0))

LoadError: AssertionError: invalid MyTime object in add_time

### 16.6 용어집

* **프로토타이핑** (prototype and patch)

    - 프로그램의 대강의 초안을 먼저 작성한 후, 테스트하면서 오류를 수정해나가는 개발 계획
```
```
* **순수 함수** (pure function)

    - 인수로 받는 객체를 수정하지 않는 함수. 대부분의 순수 함수는 반환값이 있는 유익(fruitful) 함수
```
```
* **변경자** (modifier)

    - 인수로 받는 객체 일부 또는 전체를 수정하는 함수. 대부분 변경자는 반환값이 없는 nothing인 빈 함수임
```
```
* **함수형 프로그래밍** (functional development)

    - 대부분의 함수를 순수 함수로 구현하는 프로그래밍 스타일
```
```
* **설계된 개발** (designed development)

    - 문제에 대한 높은 수준의 이해를 바탕으로, 점진적 개발이나 프로토타이핑에 비해 더 많은 계획을 하는 개발 계획
```
```
* **불변식** (invariant)

    - 프로그램 실행 중에 절대 변하지 않아야 하는 조건식
    